#(실습-2) Nori 형태소 분석기 활용

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 Elasticsearch에서 공식적으로 지원하는 한글 형태소 분석기인 Nori를 설치하고 활용해 본다. <br>
  기본 Analyzer 사용시와 Nori 형태소 분석기를 사용했을 때의 차이점에 대해 비교해 본다. <br>
2) 수강 목표
  * 한글 형태소 분석기 Nori를 노트북 환경에 설치할 수 있다.
  * Nori를 사용해서 한국어 문장에 대한 형태소 분석을 할 수 있다.
  * Nori를 활용한 색인 및 검색 명령을 수행할 수 있다.

### 실습 목차
* 1. 한글 형태소 분석기 Nori 설치
* 2. Nori analyzer의 활용
* 3. 위키데이터를 활용한 색인/검색 예시

### 데이터셋 개요
* 데이터셋: wikimedia kowiki
* 데이터셋 개요 : wikimedia에서 제공하는 한국어 데이터셋

### 환경 설정
먼저 Elasticsearch 8.8.0 버전을 설치한다.

In [ ]:
# Elasticsearch Python 패키지 설치
!pip install elasticsearch==8.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
# Elasticsearch 8.8.0 다운로드 및 압축 풀기

# 리눅스용 엘라스틱서치 서버 설치를 위한 패키지 다운로드
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz

# 코랩 노트북 환경에서 서버 구동을 위해서 PPID 1의 백그라운드 데몬 프로세스가 해당 폴더에 접근이 가능하도록 소유자 변경
!sudo chown -R daemon:daemon elasticsearch-8.8.0/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
!umount /sys/fs/cgroup
!apt install cgroup-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgroup1
The following NEW packages will be installed:
  cgroup-tools libcgroup1
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 121 kB of archives.
After this operation, 435 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcgroup1 amd64 2.0-2 [49.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cgroup-tools amd64 2.0-2 [70.8 kB]
Fetched 121 kB in 1s (225 kB/s)
Selecting previously unselected package libcgroup1:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../libcgroup1_2.0-2_amd64.deb ...
Unpacking libcgroup1:amd64 (2.0-2) ...
Selecting previously unselected package cgroup-tools.
Preparing to unpack .../cgroup-tools_2.0-2_amd64.deb ...
Unpacking cgroup-tools (2.0-2) ...
Setting up

## 1. 한글 형태소 분석기 Nori 설치


In [ ]:
# 한글 형태소 분석기 Nori 설치
! /content/elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Installed analysis-nori
-> Please restart Elasticsearch to activate any plugins installed


In [ ]:
# 플러그인 설치 확인 (analysis-nori가 보여야 함)
! /content/elasticsearch-8.8.0/bin/elasticsearch-plugin list

analysis-nori


In [ ]:
# 엘라스틱서치의 데몬 인스턴스 만들기
# Nori 설치 이전에 데몬을 생성하면 Nori가 바로 사용할 수 없음, 이때는 데몬 재실행 필요

import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(30)


In [ ]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon      1651     297 44 09:01 ?        00:00:19 /content/elasticsearch-8.8.0/jdk/bin/java -Xms4m
daemon      1775    1651 99 09:01 ?        00:00:42 /content/elasticsearch-8.8.0/jdk/bin/java -Des.n
daemon      1819    1775  0 09:01 ?        00:00:00 /content/elasticsearch-8.8.0/modules/x-pack-ml/p
root        1908     297  0 09:02 ?        00:00:00 /bin/bash -c ps -ef | grep elasticsearch
root        1910    1908  0 09:02 ?        00:00:00 grep elasticsearch


In [ ]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

******************************************************************************
Note: The 'elasticsearch-setup-passwords' tool has been deprecated. This       command will be removed in a future release.
******************************************************************************

Initiating the setup of passwords for reserved users elastic,apm_system,kibana,kibana_system,logstash_system,beats_system,remote_monitoring_user.
The passwords will be randomly generated and printed to the console.
Please confirm that you would like to continue [y/N]y


Changed password for user apm_system
PASSWORD apm_system = dL1r0v5GoAUbWXHTIV13

Changed password for user kibana_system
PASSWORD kibana_system = qyyRvBOO78m3kT1SA1W4

Changed password for user kibana
PASSWORD kibana = qyyRvBOO78m3kT1SA1W4

Changed password for user logstash_system
PASSWORD logstash_system = ddZtoyIyLVVC0bitsrfI

Changed password for user beats_system
PASSWORD beats_system = smtLQaSe7wPp4aTpfQg0

Changed password for user rem

In [ ]:
username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
password = 'awcBjNqVcIBiBUCdT93s'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': 'd96ac0383806',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'LEEowPktQXq6_ezEiPA7oQ',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## 2. Nori analyzer의 활용

In [ ]:
# Nori를 사용하지 않을 경우 형태소 분석 결과 확인
# 기본적으로는 default analyzer인 "Standard analyzer"를 사용하게 됨 (공백으로 단어 분리, 소문자로 변환, 불용어 제거, 문장부호 기호 등 제거)
import pprint

result = es.indices.analyze(text ='모든 권력은 국민으로부터 나온다.')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

result = es.indices.analyze(text ='홍대입구역너무복잡해')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

ObjectApiResponse({'tokens': [{'token': '모든', 'start_offset': 0, 'end_offset': 2, 'type': '<HANGUL>', 'position': 0}, {'token': '권력은', 'start_offset': 3, 'end_offset': 6, 'type': '<HANGUL>', 'position': 1}, {'token': '국민으로부터', 'start_offset': 7, 'end_offset': 13, 'type': '<HANGUL>', 'position': 2}, {'token': '나온다', 'start_offset': 14, 'end_offset': 17, 'type': '<HANGUL>', 'position': 3}]})
ObjectApiResponse({'tokens': [{'token': '홍대입구역너무복잡해', 'start_offset': 0, 'end_offset': 10, 'type': '<HANGUL>', 'position': 0}]})


In [ ]:
# Nori 사용할 경우 형태소 분석 결과 확인

result = es.indices.analyze(analyzer="nori", text ='모든 권력은 국민으로부터 나온다.')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

result = es.indices.analyze(analyzer="nori", text ='홍대입구역너무복잡해')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

ObjectApiResponse({'tokens': [{'token': '권력', 'start_offset': 3, 'end_offset': 5, 'type': 'word', 'position': 1}, {'token': '국민', 'start_offset': 7, 'end_offset': 9, 'type': 'word', 'position': 3}, {'token': '나오', 'start_offset': 14, 'end_offset': 17, 'type': 'word', 'position': 5}]})
ObjectApiResponse({'tokens': [{'token': '홍대', 'start_offset': 0, 'end_offset': 2, 'type': 'word', 'position': 0}, {'token': '입구', 'start_offset': 2, 'end_offset': 4, 'type': 'word', 'position': 1}, {'token': '역', 'start_offset': 4, 'end_offset': 5, 'type': 'word', 'position': 2}, {'token': '복잡', 'start_offset': 7, 'end_offset': 9, 'type': 'word', 'position': 4}]})


- nori 토크나이져 활용 시 품사를 고려하여 token화 하는 것을 알 수 있다.

## 3. 위키데이터를 활용한 색인/검색 예시

In [ ]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
!python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result


--2024-01-23 09:05:43--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74589104 (71M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’

kowiki-latest-pages 100%[===================>]  71.13M  4.19MB/s    in 17s     

2024-01-23 09:06:01 (4.18 MB/s) - ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’ saved [74589104/74589104]

Cloning into 'wikiextractor'...
remote: Enumerating objects: 771, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 771 (delta 17), reused 21 (delta 14), pack-reused 741
Receiving objects: 100% (771/771), 1.31 MiB | 2.27 MiB/s, done.
Resolving deltas: 100% (450/450), done.
INFO: Pre

In [ ]:
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장
index_docs = []
wiki_dump_json_file = '/content/extract_result/AA/wiki_00'

for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

index_docs[0]

{'id': '5',
 'revid': '36264616',
 'url': 'https://ko.wikipedia.org/wiki?curid=5',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(: James Earl Carter, Jr., 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임

In [ ]:
# 색인을 위한 mapping 설정
setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter": ["nori_posfilter"]
                }
            },
            "filter": {
                "nori_posfilter": {
                    "type": "nori_part_of_speech",
                    # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                    "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text", "analyzer": "nori"},
            "text": {"type": "text", "analyzer": "nori"}
        }
    }
}

In [ ]:
# Elasticsearch 색인/검색을 위한 공통 함수 정의

from elasticsearch import Elasticsearch, helpers
import json
import pprint as pp

def create_es_index(index, body):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, body=body)

def delete_es_index(index):
    # 지정된 인덱스 삭제
    es.indices.delete(index=index)

def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    # Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
    return helpers.bulk(es, actions)

In [ ]:
# setting으로 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", setting)

# 'test' 인덱스에 대량 색인화 수행
ret = bulk_add("test", index_docs)

# 결과 출력
print(ret)

<ipython-input-14-08a00997041c>:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index, body=body)


(61, [])


In [ ]:
# title과 text 두 필드에서 검색
body = {
    "query": {
        "multi_match": {
            "query": "대한민국 대통령",
            "fields": ['title', 'text']  # wiki에서 json만들어질때 컬럼명이 제목은 title , 내용은 text
        }
    },
    "size": 10
}
res = es.search(index="test", body=body)

<ipython-input-18-6c0981963e32>:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="test", body=body)


In [ ]:
# 결과 출력
for rst in res['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 11.246776 source:: {'id': '5', 'revid': '36264616', 'url': 'https://ko.wikipedia.org/wiki?curid=5', 'title': '지미 카터', 'text': '제임스 얼 카터 주니어(: James Earl Carter, Jr., 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들

#Reference